In [15]:
# !pip install "sagemaker>=2.48.0" "transformers==4.6.1" "datasets[s3]==1.6.2" --upgrade
# !pip install sagemaker transformers 
# !pip install torch

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [10]:
import sagemaker

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::448807757624:role/service-role/AmazonSageMaker-ExecutionRole-20211202T101582
sagemaker bucket: sagemaker-us-east-2-448807757624
sagemaker session region: us-east-2


In [3]:
%%capture
import IPython
!conda install -c conda-forge ipywidgets -y
IPython.Application.instance().kernel.do_shutdown(True) # has to restart kernel so changes are used

In [5]:
from datasets import load_dataset
from transformers import AutoTokenizer

s3_prefix = 'datasets/xsum_filtered'

xsum_filtered_train = load_dataset("data/xsum_filtered/train")
xsum_filtered_val= load_dataset("data/xsum_filtered/val")
xsum_filtered_test= load_dataset("data/xsum_filtered/test")

Using custom data configuration default


Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499. Subsequent calls will reuse this data.


In [24]:
xsum_train = load_dataset("xsum", split="train")
xsum_eval = load_dataset("xsum", split="validation")

Using custom data configuration default
Reusing dataset xsum (/root/.cache/huggingface/datasets/xsum/default/1.1.0/128741c17b7a4c939dbf844a75a5e83deadd07deaf4b2eda2056ed8eebdb03ae)
Using custom data configuration default
Reusing dataset xsum (/root/.cache/huggingface/datasets/xsum/default/1.1.0/128741c17b7a4c939dbf844a75a5e83deadd07deaf4b2eda2056ed8eebdb03ae)


In [ ]:
xsum_filtered_train = xsum_filtered_train.filter(lambda x: x['keep']).remove_columns('keep')
xsum_filtered_val= xsum_filtered_train.filter(lambda x: x['keep']).remove_columns('keep')
xsum_filtered_test= xsum_filtered_train.filter(lambda x: x['keep']).remove_columns('keep')

In [6]:
"""
Download Data: this takes a while for some reason.
"""

column_names = xsum_train.column_names
text_column, summary_column = "document", "summary"

Using custom data configuration default
Reusing dataset xsum (/root/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499)
Using custom data configuration default
Reusing dataset xsum (/root/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499)


In [7]:
"""
Process Data
"""

MODEL_NAME = 'facebook/bart-large'
max_source_length = 1024
max_target_length = 128
padding=False

tokenizer = AutoTokenizer.from_pretrained(
MODEL_NAME, cache_dir=None, use_fast=True, revision="main", use_auth_token=False,
)

sep_token = '</s>'

def preprocess_function(examples):

    inputs = examples[text_column]
    targets = examples[summary_column]
    
    # Tokenize Input
    model_inputs = tokenizer(
        text = inputs, text_pair=targets, max_length=max_source_length, 
        padding=padding, truncation='only_first', add_special_tokens=True
    )
    
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets, max_length=max_target_length, padding=padding, truncation=True
        )

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label]
            for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    
    return model_inputs

train_dataset = xsum_filtered_train
train_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=None,
    remove_columns=column_names,
    load_from_cache_file=False,
)

val_dataset = xsum_filtered_val
val_dataset = val_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=None,
    remove_columns=column_names,
    load_from_cache_file=False,
)

In [8]:
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

In [11]:
import botocore
from datasets.filesystems import S3FileSystem

s3 = S3FileSystem()  

# save train_dataset to s3
training_input_path = f's3://{sess.default_bucket()}/{s3_prefix}/train'
train_dataset.save_to_disk(training_input_path,fs=s3)

# save test_dataset to s3
test_input_path = f's3://{sess.default_bucket()}/{s3_prefix}/test'
val_dataset.save_to_disk(test_input_path,fs=s3)